<a href="https://colab.research.google.com/github/rohhj622/windPowerPrediction/blob/master/code1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from keras import models, layers, Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Input

## 1) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 3) 데이터 불러오기 시작

In [ ]:
allData = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/preprocessing_Data.csv') # 컬럼명 숫자로 들어가게
allData = allData.drop('Unnamed: 0',1)
allData

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01 00:00:00,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01 00:00:00,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01 00:00:00,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01 00:00:00,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01 00:00:00,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2018-12-31 19:00:00,2018-12-31 00:00:00,19:00,8699,4.3,4.7,340,58.0,4.8,-3.2,1032.1,1034.7,3.3
52580,2018-12-31 20:00:00,2018-12-31 00:00:00,20:00,10694,3.9,4.5,360,61.0,4.9,-2.9,1032.0,1034.6,2.9
52581,2018-12-31 21:00:00,2018-12-31 00:00:00,21:00,5494,3.5,4.0,360,59.0,4.6,-3.7,1031.9,1034.5,1.9
52582,2018-12-31 22:00:00,2018-12-31 00:00:00,22:00,2718,3.8,3.4,20,61.0,4.9,-3.0,1031.6,1034.2,2.1


# 2. 데이터 가공하기 

## 1) datetime 컬럼 제작, datetime 형식으로 변환

In [ ]:
df = allData.copy()
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M', errors='ignore') #합친거 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d', errors='ignore')  #date형식으로 변환
#df['time'] = pd.to_datetime(df['time'],format='%H:%M', errors='ignore') #time 형식으로 변환
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1


## 2) 년(year), 월(month), 일(day), 계절(season) 컬럼 생성



In [ ]:
df['year'] = df['date'].dt.year # 년
df['month'] = df['date'].dt.month # 월
df['day'] = df['date'].dt.day# 일

conditionlist = [
    (df['month'] == 3)|(df['month'] == 4)| (df['month'] == 5) ,
    (df['month'] == 6)|(df['month'] == 7)|(df['month'] == 8) ,
    (df['month'] == 9)|(df['month'] == 10)|(df['month'] == 11),
    (df['month'] == 12)|(df['month'] == 1)|(df['month'] == 2) ]
choicelist = ['0', '1', '2','3']
#choicelist = ['spring', 'summer', 'autumn','winter']
df['season'] = np.select(conditionlist, choicelist, default='Not Specified')
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem,year,month,day,season
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0,2013,1,1,3
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0,2013,1,1,3
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0,2013,1,1,3
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3,2013,1,1,3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1,2013,1,1,3


# 4. 훈련 & 예측

## 1) train-test data 나누기

In [ ]:
df2 = df.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          52584 non-null  datetime64[ns]
 1   date              52584 non-null  datetime64[ns]
 2   time              52584 non-null  object        
 3   windForce         52584 non-null  int64         
 4   tem               52584 non-null  float64       
 5   windSpeed         52584 non-null  float64       
 6   windDirect        52584 non-null  int64         
 7   humidity          52584 non-null  float64       
 8   vaporPressure     52584 non-null  float64       
 9   dewPoint          52584 non-null  float64       
 10  pressure          52584 non-null  float64       
 11  seaLevelPressure  52584 non-null  float64       
 12  groundTem         52584 non-null  float64       
 13  year              52584 non-null  int64         
 14  month             5258

In [ ]:
X = df2[['tem','windSpeed','windDirect','humidity']] #,'groundTem','seaLevelPressure'
y = df2[['windForce']]

transform = MinMaxScaler()
trans_X=transform.fit_transform(X)
trans_y=transform.fit_transform(y)

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)
print('train data 개수 : ', len(X_train))
print('test data 개수: ', len(X_test))
print(len(X_train)+len(X_test))

train data 개수 :  42067
test data 개수:  10517
52584


## 5. test


In [ ]:
network = models.Sequential()

In [ ]:
network.add(layers.Dense(128, input_dim=4,activation='relu'))
network.add(layers.Dense(64,activation='relu'))
network.add(layers.Dense(1))

In [ ]:
network.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
# optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc']
# optimizer='adam', metrics=['accuracy']
# loss='binary_crossentropy',metrics=['binary_accuracy']

In [ ]:
network.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
2630/2630 [==============================] - 7s 3ms/step - loss: 47590.3867 - acc: 0.1186
Epoch 2/10
2630/2630 [==============================] - 6s 2ms/step - loss: 47590.3164 - acc: 0.1186
Epoch 3/10
2431/2630 [==========================>...] - ETA: 0s - loss: 47574.9805 - acc: 0.1180

In [ ]:
y_pred = network.predict(X_test).flatten()

In [ ]:
plt.figure(figsize=(16,10))
ax1 = sns.distplot(y_test,hist=False,label='y')
ax2 = sns.distplot(y_pred,hist=False,label='y_hat',ax=ax1)

plt.show()
plt.close()

In [ ]:
df2.head()

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem,year,month,day,season
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0,2013,1,1,3
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0,2013,1,1,3
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0,2013,1,1,3
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3,2013,1,1,3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1,2013,1,1,3


In [ ]:
# X = df2[['tem','windSpeed','humidity','groundTem','windDirect','seaLevelPressure']] #,'windDirect'
# X = df2[['tem','windSpeed','groundTem']] #,'windDirect'
transform = MinMaxScaler()

scale_cols=['tem','windSpeed','groundTem','windForce']
df_scaled = df2[['tem','windSpeed','groundTem','windForce']] 
df_scaled = transform.fit_transform(df_scaled)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

,tem,windSpeed,groundTem,windForce
0,0.190123,0.180791,0.061135,0.000000
1,0.180247,0.146893,0.061135,0.000000
2,0.195062,0.169492,0.061135,0.007663
3,0.224691,0.096045,0.065502,0.022407
4,0.172840,0.101695,0.062591,0.045065
...,...,...,...,...
52579,0.264198,0.265537,0.109170,0.724615
52580,0.254321,0.254237,0.103348,0.890796
52581,0.244444,0.225989,0.088792,0.457643
52582,0.251852,0.192090,0.091703,0.226406


In [ ]:
WINDOW_SIZE = 100

trans_X = df_scaled[['tem','windSpeed','groundTem']]
trans_y = df_scaled[['windForce']]

X_train, X_test,y_train, y_test = train_test_split(trans_X,trans_y,test_size=0.2,random_state=10)

In [ ]:
def make_dataset(data, label, window_size=1):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

train_feature, train_label = make_dataset(trans_X, trans_y, 1)
train_feature

array([[[0.19012346, 0.18079096, 0.06113537]],

       [[0.18024691, 0.14689266, 0.06113537]],

       [[0.19506173, 0.16949153, 0.06113537]],

       ...,

       [[0.25432099, 0.25423729, 0.10334789]],

       [[0.24444444, 0.2259887 , 0.08879185]],

       [[0.25185185, 0.1920904 , 0.09170306]]])

In [ ]:
#X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)
# X_train, X_test,y_train, y_test = train_test_split(trans_X,trans_y,test_size=0.2,random_state=10)
X_train, X_test,y_train, y_test = train_test_split(train_feature,train_label,test_size=0.2,random_state=10)


print('train data 개수 : ', len(X_train))
print('test data 개수: ', len(X_test))
print(len(X_train)+len(X_test))

train data 개수 :  42066
test data 개수:  10517
52583


In [ ]:
X_train.head()

AttributeError: ignored

In [ ]:
X_train = X_train.values
X_test= X_test.values

y_train = y_train.values
y_test = y_test.values

In [ ]:
X_train_t = X_train.reshape(X_train.shape[0], 3, 1) 
X_test_t = X_test.reshape(X_test.shape[0], 3, 1)

In [ ]:
# from keras.layers import LSTM 
# from keras.models import Sequential 
# from keras.layers import Dense 
# import keras.backend as K 
# from keras.callbacks import EarlyStopping

# K.clear_session()
    
# model = Sequential() # Sequeatial Model 
# model.add(LSTM(20, input_shape=(3, 1), return_sequences=True)) # (timestep, feature) 
# model.add(Dense(1)) # output = 1 
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
# model.summary()

In [ ]:
# early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

# model.fit(X_train_t, y_train, epochs=100,batch_size=30, verbose=1, callbacks=[early_stop])

In [ ]:
network = models.Sequential()
network.add(LSTM(20, input_shape=(3, 1), return_sequences=True))

# network.add(layers.Dense(2, input_dim=3, activation='relu'))
# network.add(layers.LSTM(2, input_shape=X_train.shape, return_sequences=False))
# network.add(layers.Dense(2))
# network.add(layers.Dense(1))

network.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])

network.summary()

In [ ]:
network.fit(X_train_t, y_train, epochs=43 , batch_size=32)


scores = network.evaluate(X_test, y_test, batch_size=32)
print("%s: %.2f%%" %(network.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = network.predict(X_test).flatten()

plt.figure(figsize=(16,10))
ax1 = sns.distplot(y_test,hist=False,label='y')
ax2 = sns.distplot(y_pred,hist=False,label='y_hat',ax=ax1)

plt.show()
plt.close()